> ### EEE2020: Data Structures & Algorithms

# Lecture 11: Graphs

## 1. Graph ADT

In [1]:
import sys

In [2]:
class Vertex:
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}
        self.color = 'white'
        self.dist = sys.maxsize
        self.pred = None
        self.disc = 0
        self.fin = 0
    
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight
        
    def setColor(self,color):
        self.color = color
        
    def setDistance(self,d):
        self.dist = d

    def setPred(self,p):
        self.pred = p

    def setDiscovery(self,dtime):
        self.disc = dtime
        
    def setFinish(self,ftime):
        self.fin = ftime
        
    def getFinish(self):
        return self.fin
        
    def getDiscovery(self):
        return self.disc
        
    def getPred(self):
        return self.pred
        
    def getDistance(self):
        return self.dist
        
    def getColor(self):
        return self.color
    
    def getConnections(self):
        return self.connectedTo.keys()

    def getId(self):
        return self.id
    
    def getWeight(self,nbr):
        return self.connectedTo[nbr]

    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])    


In [3]:
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self,n):
        return n in self.vertList

    def addEdge(self,f,t,weight=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], weight)

    def getVertices(self):
        return self.vertList.keys()

    def __iter__(self):
        return iter(self.vertList.values())

In [4]:
g = Graph()

In [5]:
for i in range(6):
    g.addVertex(i)

In [6]:
g.vertList

{0: <__main__.Vertex at 0x280d76649d0>,
 1: <__main__.Vertex at 0x280d76647f0>,
 2: <__main__.Vertex at 0x280d7676eb0>,
 3: <__main__.Vertex at 0x280d7676f10>,
 4: <__main__.Vertex at 0x280d7676f70>,
 5: <__main__.Vertex at 0x280d7676fa0>}

In [7]:
g.addEdge(0,1,5)
g.addEdge(0,5,2)
g.addEdge(1,2,4)
g.addEdge(2,3,9)
g.addEdge(3,4,7)
g.addEdge(3,5,3)
g.addEdge(4,0,1)
g.addEdge(5,4,8)
g.addEdge(5,2,1)

In [8]:
for v in g:
    for w in v.getConnections():
        print("( %s , %s )" % (v.getId(), w.getId()))

( 0 , 1 )
( 0 , 5 )
( 1 , 2 )
( 2 , 3 )
( 3 , 4 )
( 3 , 5 )
( 4 , 0 )
( 5 , 4 )
( 5 , 2 )


In [9]:
for v in g:
    print(v)

0 connectedTo: [1, 5]
1 connectedTo: [2]
2 connectedTo: [3]
3 connectedTo: [4, 5]
4 connectedTo: [0]
5 connectedTo: [4, 2]


## 2. Breadth First Search (BFS): Word Ladder Problem

In [10]:
from queues import Queue

In [11]:
def buildGraph(wordFile):
    d = {}
    g = Graph()
    wfile = open(wordFile,'r')
    # create buckets of words that differ by one letter
    for line in wfile:
        word = line[:-1]
        for i in range(len(word)):
            bucket = word[:i] + '_' + word[i+1:]
            if bucket in d:
                d[bucket].append(word)
            else:
                d[bucket] = [word]
    # add vertices and edges for words in the same bucket
    for bucket in d.keys():
        for word1 in d[bucket]:
            for word2 in d[bucket]:
                if word1 != word2:
                    g.addEdge(word1,word2)
    return g

In [12]:
def bfs(g,start):
    start.setDistance(0)
    start.setPred(None)
    vertQueue = Queue()
    vertQueue.enqueue(start)
    while (vertQueue.size() > 0):
        currentVert = vertQueue.dequeue()
        for nbr in currentVert.getConnections():
            if (nbr.getColor() == 'white'):
                nbr.setColor('gray')
                nbr.setDistance(currentVert.getDistance() + 1)
                nbr.setPred(currentVert)
                vertQueue.enqueue(nbr)
        currentVert.setColor('black')

In [13]:
def traverse(y):
    x = y
    while (x.getPred()):
        print(x.getId())
        x = x.getPred()
    print(x.getId())

In [14]:
wordgraph = buildGraph("fourletterwords.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'fourletterwords.txt'

In [15]:
bfs(wordgraph, wordgraph.getVertex('FOOL'))

NameError: name 'wordgraph' is not defined

In [16]:
traverse(wordgraph.getVertex('SAGE'))

NameError: name 'wordgraph' is not defined

## 2. Depth First Search (DFS): The Knight's Tour Problem

In [17]:
def knightGraph(bdSize):
    ktGraph = Graph()
    for row in range(bdSize):
        for col in range(bdSize):
            nodeId = posToNodeId(row,col,bdSize)
            newPositions = genLegalMoves(row,col,bdSize)
            for e in newPositions:
                nid = posToNodeId(e[0],e[1],bdSize)
                ktGraph.addEdge(nodeId,nid)
    return ktGraph

def posToNodeId(row, column, board_size):
    return (row * board_size) + column

In [18]:
def genLegalMoves(x,y,bdSize):
    newMoves = []
    moveOffsets = [(-1,-2),(-1,2),(-2,-1),(-2,1),
                   ( 1,-2),( 1,2),( 2,-1),( 2,1)]
    for i in moveOffsets:
        newX = x + i[0]
        newY = y + i[1]
        if legalCoord(newX,bdSize) and \
                        legalCoord(newY,bdSize):
            newMoves.append((newX,newY))
    return newMoves

def legalCoord(x,bdSize):
    if x >= 0 and x < bdSize:
        return True
    else:
        return False

In [19]:
def knightTour(n,path,u,limit):
        u.setColor('gray')
        path.append(u)
        if n < limit:
            nbrList = list(u.getConnections())
            i = 0
            done = False
            while i < len(nbrList) and not done:
                if nbrList[i].getColor() == 'white':
                    done = knightTour(n+1, path, nbrList[i], limit)
                i = i + 1
            if not done:  # prepare to backtrack
                path.pop()
                u.setColor('white')
        else:
            done = True
        return done

In [20]:
def orderByAvail(n):
    resList = []
    for v in n.getConnections():
        if v.getColor() == 'white':
            c = 0
            for w in v.getConnections():
                if w.getColor() == 'white':
                    c = c + 1
            resList.append((c,v))
    resList.sort(key=lambda x: x[0])
    return [y[1] for y in resList]
    
def knightTourBetter(n,path,u,limit):  #use order by available function
        u.setColor('gray')
        path.append(u)
        if n < limit:
            nbrList = orderByAvail(u) #기존 nightTour와 이 부분만 딱 다르다
            i = 0
            done = False
            while i < len(nbrList) and not done:
                if nbrList[i].getColor() == 'white':
                    done = knightTour(n+1, path, nbrList[i], limit)
                i = i + 1
            if not done:  # prepare to backtrack
                path.pop()
                u.setColor('white')
        else:
            done = True
        return done

In [21]:
kg = knightGraph(6)  

In [22]:
thepath = []

In [23]:
start = kg.getVertex(4)

In [24]:
knightTour(0,thepath,start,35)

True

In [25]:
for v in thepath:
    print(v.getId())

4
8
0
13
9
1
12
20
24
32
28
17
21
10
2
6
19
30
26
34
23
15
7
3
11
22
35
27
31
18
14
25
33
29
16
5


In [26]:
knightTourBetter(0,thepath,start,35)
for v in thepath:
    print(v.getId())

4
8
0
13
9
1
12
20
24
32
28
17
21
10
2
6
19
30
26
34
23
15
7
3
11
22
35
27
31
18
14
25
33
29
16
5


## 3. General DFS

In [30]:
class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0

    def dfs(self):
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)

    def dfsvisit(self,startVertex):
        startVertex.setColor('gray')
        self.time += 1
        startVertex.setDiscovery(self.time)
        for nextVertex in startVertex.getConnections():
            if nextVertex.getColor() == 'white':
                nextVertex.setPred(startVertex)
                self.dfsvisit(nextVertex)
        startVertex.setColor('black')
        self.time += 1
        startVertex.setFinish(self.time)
        
    def print_time(self):
        for key in sorted(list(self.getVertices())):
            print(str(key) + ": " + str(self.getVertex(key).getDiscovery()) + "/" + str(self.getVertex(key).getFinish()))

In [31]:
graph = DFSGraph()

myVertices = ['A','B','C','D','E','F']
# add vertices
for i in range(len(myVertices)):
    graph.addVertex(myVertices[i])

graph.addEdge('A', 'B')
graph.addEdge('A', 'D')
graph.addEdge('B', 'C')
graph.addEdge('B', 'D')
graph.addEdge('D', 'E')
graph.addEdge('E', 'B')
graph.addEdge('E', 'F')
graph.addEdge('F', 'C')

graph.dfs()

In [32]:
graph.print_time()

A: 1/12
B: 2/11
C: 3/4
D: 5/10
E: 6/9
F: 7/8


## 4. Dijkstra's Algorithm

In [33]:
class PriorityQueue:
    def __init__(self):
        self.heapArray = [(0,0)]
        self.currentSize = 0

    def buildHeap(self,alist):
        self.currentSize = len(alist)
        self.heapArray = [(0,0)]
        for i in alist:
            self.heapArray.append(i)
        i = len(alist) // 2            
        while (i > 0):
            self.percDown(i)
            i = i - 1
                        
    def percDown(self,i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapArray[i][0] > self.heapArray[mc][0]:
                tmp = self.heapArray[i]
                self.heapArray[i] = self.heapArray[mc]
                self.heapArray[mc] = tmp
            i = mc
                
    def minChild(self,i):
        if i*2 > self.currentSize:
            return -1
        else:
            if i*2 + 1 > self.currentSize:
                return i*2
            else:
                if self.heapArray[i*2][0] < self.heapArray[i*2+1][0]:
                    return i*2
                else:
                    return i*2+1

    def percUp(self,i):
        while i // 2 > 0:
            if self.heapArray[i][0] < self.heapArray[i//2][0]:
                tmp = self.heapArray[i//2]
                self.heapArray[i//2] = self.heapArray[i]
                self.heapArray[i] = tmp
            i = i//2
 
    def add(self,k):
        self.heapArray.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)

    def delMin(self):
        retval = self.heapArray[1][1]
        self.heapArray[1] = self.heapArray[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapArray.pop()
        self.percDown(1)
        return retval
        
    def isEmpty(self):
        if self.currentSize == 0:
            return True
        else:
            return False

    def decreaseKey(self,val,amt):
        done = False
        i = 1
        myKey = 0
        while not done and i <= self.currentSize:
            if self.heapArray[i][1] == val: #self.currentSize범위 내에서 원하는 값을 찾을 때까지
                done = True #원하는 값을 찾으면 while문 중지하고
                myKey = i #mykey=i로 설정
            else:
                i = i + 1 #i를 하나씩 늘려감
        if myKey > 0:
            self.heapArray[myKey] = (amt,self.heapArray[myKey][1]) #vertex에 대한 거리정보 수정
            self.percUp(myKey) #percUp함.
            
    def __contains__(self,vtx):
        for pair in self.heapArray:
            if pair[1] == vtx:
                return True
        return False

In [34]:
theHeap = PriorityQueue()
theHeap.add((2,'x'))
theHeap.add((3,'y'))
theHeap.add((5,'z'))
theHeap.add((6,'a'))
theHeap.add((4,'d'))

In [35]:
theHeap.currentSize

5

In [36]:
theHeap.delMin()

'x'

In [37]:
theHeap.delMin()

'y'

In [38]:
theHeap.decreaseKey('z',1)

In [39]:
theHeap.heapArray

[(0, 0), (1, 'z'), (6, 'a'), (4, 'd')]

In [40]:
theHeap.delMin()

'z'

In [41]:
def dijkstra(aGraph,start):
    pq = PriorityQueue()
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in aGraph])
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            newDist = currentVert.getDistance() \
                    + currentVert.getWeight(nextVert)
            if newDist < nextVert.getDistance():
                nextVert.setDistance( newDist )
                nextVert.setPred(currentVert)
                pq.decreaseKey(nextVert,newDist)

        if currentVert == start:        
            print("Start node: %s" % currentVert.getId())
        else:    
            print("%s --> %s: distance = %s" % (currentVert.getId(), currentVert.getPred().id, currentVert.getDistance()))

In [42]:
graph = Graph()
graph.addVertex('u')
graph.addVertex('v')
graph.addVertex('w')
graph.addVertex('x')
graph.addVertex('y')
graph.addVertex('z')

graph.addEdge('u', 'v', 2)
graph.addEdge('u', 'x', 1)
graph.addEdge('v', 'x', 2)
graph.addEdge('v', 'w', 3)
graph.addEdge('u', 'w', 5)
graph.addEdge('x', 'w', 3)
graph.addEdge('x', 'y', 1)
graph.addEdge('y', 'w', 1)
graph.addEdge('y', 'z', 1)
graph.addEdge('w', 'z', 5)

graph.addEdge('v', 'u', 2)
graph.addEdge('x', 'u', 1)
graph.addEdge('x', 'v', 2)
graph.addEdge('w', 'v', 3)
graph.addEdge('w', 'u', 5)
graph.addEdge('w', 'x', 3)
graph.addEdge('y', 'x', 1)
graph.addEdge('w', 'y', 1)
graph.addEdge('z', 'y', 1)
graph.addEdge('z', 'w', 5)


dijkstra(graph, graph.getVertex('u'))

Start node: u
x --> u: distance = 1
v --> u: distance = 2
y --> x: distance = 2
w --> y: distance = 3
z --> y: distance = 3


## 5. Prim's Algorithm

In [43]:
def prim(G,start): #최소신장트리
    pq = PriorityQueue()
    for v in G:
        v.setDistance(sys.maxsize)
        v.setPred(None)
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in G])
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            newCost = currentVert.getWeight(nextVert)
            if nextVert in pq and newCost<nextVert.getDistance():
                nextVert.setPred(currentVert)
                nextVert.setDistance(newCost)
                pq.decreaseKey(nextVert,newCost)

        if currentVert == start:        
            print("Start node: %s" % currentVert.getId())
        else:                
            print("%s --> %s" % (currentVert.getId(), currentVert.getPred().id))

In [44]:
graph = Graph()
graph.addVertex('A')
graph.addVertex('B')
graph.addVertex('C')
graph.addVertex('D')
graph.addVertex('E')
graph.addVertex('F')
graph.addVertex('G')

graph.addEdge('A', 'B', 2)
graph.addEdge('A', 'C', 3)
graph.addEdge('B', 'C', 1)
graph.addEdge('B', 'D', 1)
graph.addEdge('B', 'E', 4)
graph.addEdge('D', 'E', 1)
graph.addEdge('C', 'F', 5)
graph.addEdge('E', 'F', 1)
graph.addEdge('F', 'G', 1)

graph.addEdge('B', 'A', 2)
graph.addEdge('C', 'A', 3)
graph.addEdge('C', 'B', 1)
graph.addEdge('D', 'B', 1)
graph.addEdge('E', 'B', 4)
graph.addEdge('E', 'D', 1)
graph.addEdge('F', 'C', 5)
graph.addEdge('F', 'E', 1)
graph.addEdge('G', 'F', 1)

prim(graph, graph.getVertex('A'))

Start node: A
B --> A
C --> B
D --> B
E --> D
F --> E
G --> F


In [48]:
def prim(G,start):
    ### CODE HERE ###
    pq=PriorityQueue()
    for v in G:
        v.setDistance(sys.maxsize)
        v.setPred(None)
    G.getVertex(start).setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in G])
    while not pq.isEmpty():
        currentVert=pq.delMin()
        for nextVert in currentVert.getConnections():
            newCost=currentVert.getWeight(nextVert)
            if nextVert in pq and newCost<nextVert.getDistance():
                nextVert.setPred(currentVert)
                nextVert.setDistance(newCost)
                pq.decreaseKey(newVert,newCost)

In [50]:
g = Undirected_DFSGraph()

myVertices = ['A','B','C','D','E','F']
for alpha in myVertices:
    print(alpha, 'is connected to')
    for k, v in prim(g,'A').getVertex(alpha).connectedTo.items():
        print(k.getId(), ':', v)
    print('-' * 20)

NameError: name 'Undirected_DFSGraph' is not defined